In [1]:
import os

assert os.environ['CONDA_DEFAULT_ENV']=='ttv'

## CTOIS

In [2]:
import sys
sys.path.insert(0, '../code')
from utils import *

In [3]:
df2 = get_ctois()
df2.head()

,TIC ID,CTOI,Promoted to TOI,MASTER,SG1A,SG1B,SG2,SG3,SG4,SG5,...,Stellar Eff Temp (K) err,Stellar log(g) (cm/s^2),Stellar log(g) (cm/s^2) err,Stellar Radius (R_Sun),Stellar Radius (R_Sun) err,CTOI lastmod,User,Group,Tag,Notes
0,17361,17361.01,3127.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,121.637,4.25352,0.078445,1.25819,0.062226,2020-08-10 07:20:18,montalto,NaN,29419,"new CTOI, Montalto et al. (2020)"
1,160363,160363.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,129.081,4.15460,0.093692,1.66309,0.075980,2021-02-11 04:13:20,steuer,NaN,94179,Found with our in-house single transit finder
2,592503,592503.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,70.000,NaN,NaN,3.23242,NaN,2022-10-20 09:02:36,montalto,NaN,420769,"new CTOI, Montalto (2022)"
3,769986,769986.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,122.000,4.20725,NaN,1.04351,NaN,2022-10-20 09:04:55,montalto,NaN,421495,"new CTOI, Montalto (2022)"
4,991806,991806.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,17.480,NaN,NaN,3.63923,NaN,2022-10-20 09:02:36,montalto,NaN,420793,"new CTOI, Montalto (2022)"


In [4]:
nsys2 = df2['TIC ID'].value_counts()
nsys2

64837857      4
206544316     4
92349924      3
300903537     3
100504488     3
             ..
159059181     1
159098316     1
159159904     1
159185219     1
2050290974    1
Name: TIC ID, Length: 2707, dtype: int64

In [5]:
tics_multis2 = list(nsys2[nsys2>1].index)
print(len(tics_multis2))

136


In [21]:
ttv_1st_order2 = []       # commensurability in 1 planet pair 
ttv_2nd_order2 = []
ttv_chain2 = []           # more than 1 planet pair
deltas1 = []
deltas2 = []
texts = []
for tic in tics_multis2:
    dd = df2[df2['TIC ID']==tic].squeeze()
    periods = dd["Period (days)"].values
    Nplanets = len(periods)
    
    resonances_1st_order, delta1 = get_resonant_pairs(periods, order=1, tol=0.01);
    resonances_2nd_order, delta2 = get_resonant_pairs(periods, order=2, tol=0.01);
    msg = f"TIC {tic} (nplanets={Nplanets}): <br>\n"
    show = False
    if len(resonances_1st_order)>=1:
        msg += f"First Order: {' '.join(resonances_1st_order)}\n"
        ttv_1st_order2.append(tic)
        deltas1.append(min(delta1))
        if len(resonances_1st_order)>=2:
            ttv_chain2.append(tic)
        show = True
    if len(resonances_2nd_order)>=1:
        msg += f"Second Order: {' '.join(resonances_2nd_order)}\n"
        ttv_2nd_order2.append(tic)
        deltas2.append(min(delta2))
        show = True
    if show:
        print(msg)
        texts.append(msg)

TIC 204497617 (nplanets=2): <br>
First Order: 2:1 | P=(3.18,1.59) n=(1,2) (delta=0.02%)
Second Order: 4:2 | P=(3.18,1.59) n=(1,2) (delta=0.02%)

TIC 29478974 (nplanets=2): <br>
First Order: 2:1 | P=(24.90,12.46) n=(1,2) (delta=0.06%)
Second Order: 4:2 | P=(24.90,12.46) n=(1,2) (delta=0.06%)

TIC 207080350 (nplanets=2): <br>
First Order: 2:1 | P=(18.38,9.11) n=(1,2) (delta=0.94%)
Second Order: 4:2 | P=(18.38,9.11) n=(1,2) (delta=0.94%)

TIC 347332255 (nplanets=2): <br>
First Order: 3:2 | P=(13.67,9.11) n=(1,2) (delta=0.02%)
Second Order: 6:4 | P=(13.67,9.11) n=(1,2) (delta=0.02%)

TIC 253922296 (nplanets=2): <br>
Second Order: 3:1 | P=(9.43,3.14) n=(1,2) (delta=0.03%)



In [8]:
import pandas as pd

s1 = pd.DataFrame(np.c_[ttv_1st_order2,deltas1], columns=['tic','delta_min'])
s1['1st order'] = True
s1.sort_values(by='delta_min')

,tic,delta_min,1st order
0,204497617.0,0.000219,True
3,347332255.0,0.000219,True
1,29478974.0,0.000557,True
2,207080350.0,0.009380,True


In [9]:
s2 = pd.DataFrame(np.c_[ttv_2nd_order2,deltas2], columns=['tic','delta_min'])
s2['2nd order'] = True
s2.sort_values(by='delta_min')

,tic,delta_min,2nd order
0,204497617.0,0.000219,True
3,347332255.0,0.000219,True
4,253922296.0,0.000295,True
1,29478974.0,0.000557,True
2,207080350.0,0.009380,True


In [17]:
res2 = pd.merge(s1, s2, how='outer')
res2 = res2.replace(np.nan, False)
res2['tic'] = res2.tic.map(int)
res2 = res2.sort_values(by='delta_min')
res2

,tic,delta_min,1st order,2nd order
0,204497617,0.000219,True,True
3,347332255,0.000219,True,True
4,253922296,0.000295,False,True
1,29478974,0.000557,True,True
2,207080350,0.009380,True,True


In [18]:
res2.to_csv('../data/tics_resonance_from_ctoi.csv')

In [24]:
len(texts), len(res2)

(5, 5)

In [23]:
idx = res2.index.values
for i in idx:
    print(texts[i])

TIC 204497617 (nplanets=2): <br>
First Order: 2:1 | P=(3.18,1.59) n=(1,2) (delta=0.02%)
Second Order: 4:2 | P=(3.18,1.59) n=(1,2) (delta=0.02%)

TIC 347332255 (nplanets=2): <br>
First Order: 3:2 | P=(13.67,9.11) n=(1,2) (delta=0.02%)
Second Order: 6:4 | P=(13.67,9.11) n=(1,2) (delta=0.02%)

TIC 253922296 (nplanets=2): <br>
Second Order: 3:1 | P=(9.43,3.14) n=(1,2) (delta=0.03%)

TIC 29478974 (nplanets=2): <br>
First Order: 2:1 | P=(24.90,12.46) n=(1,2) (delta=0.06%)
Second Order: 4:2 | P=(24.90,12.46) n=(1,2) (delta=0.06%)

TIC 207080350 (nplanets=2): <br>
First Order: 2:1 | P=(18.38,9.11) n=(1,2) (delta=0.94%)
Second Order: 4:2 | P=(18.38,9.11) n=(1,2) (delta=0.94%)



In [16]:
np.savetxt('../data/tics_resonance_from_ctoi.txt', res2.tic.tolist(), fmt="%d")
np.savetxt('../data/tics_resonance_from_ctoi_1st_order.txt', ttv_1st_order2, fmt="%d")
np.savetxt('../data/tics_resonance_from_ctoi_2nd_order.txt', ttv_2nd_order2, fmt="%d")